In [83]:
import torch
import torch.nn as nn

class CrossedAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(1024, 256, 1, bias=False)
        self.k_conv = nn.Conv1d(1024, 256, 1, bias=False)
        self.v_conv = nn.Conv1d(1024, 256, 1, bias=False)
        self.trans_conv = nn.Conv1d(256, 256, 1)
        self.after_norm = nn.BatchNorm1d(256)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q_tensor, kv_tensor):
        x_q = self.q_conv(q_tensor.permute(0, 2, 1))    
        x_k = self.k_conv(kv_tensor.permute(0, 2, 1))  
        x_v = self.v_conv(kv_tensor.permute(0, 2, 1))
        print('x_q: ', x_q.shape)
        energy = torch.matmul(x_q.permute(0, 2, 1), x_k) nn.Softmax
        print('energy shape: ', energy.shape)
        attention = self.softmax(energy)
        # attention = attention / (1e-9 + attention.sum(dim=1, keepdims=True))
        x_r = torch.matmul(attention, x_v.permute(0, 2, 1))  
        print('x_rs shape: ', x_r.shape)
        
        res = (x_q - x_r.permute(0, 2, 1))
        print('res shape: ', res.shape)
        
        x_r = self.act(self.after_norm(self.trans_conv(res)))
        print('result shape: ', x_r.shape)
        
        return x_r.permute(0, 2, 1)
    
ca = CrossedAttention(1024)

In [84]:
rand_q = torch.ones((4, 1, 1024))
rand_kv = torch.ones((4, 6, 1024))

result = ca(rand_q, rand_kv)

x_q:  torch.Size([4, 256, 1])
energy shape:  torch.Size([4, 1, 6])
x_rs shape:  torch.Size([4, 1, 256])
res shape:  torch.Size([4, 256, 1])
result shape:  torch.Size([4, 256, 1])


In [85]:
result.shape

torch.Size([4, 1, 256])

In [50]:
rand_q = torch.ones((4, 1, 1024))
rand_kv = torch.ones((4, 6, 1024))

result = ca(rand_q, rand_kv)

x shape:  torch.Size([4, 1, 1024])
x_q:  torch.Size([4, 1024, 1])
x_v:  torch.Size([4, 1024, 6])
energy:  torch.Size([4, 1, 6])
attention:  torch.Size([4, 1, 6])
x_r:  torch.Size([4, 1, 1024])
res:  torch.Size([4, 1024, 1])


In [51]:
result.shape

torch.Size([4, 1, 1024])

In [16]:
softmax = nn.Softmax(dim=-1)
attention = softmax(energy)
attention.shape

torch.Size([4, 1, 6])

In [18]:
x_v = x_q
x_r = torch.matmul(attention, x_v.permute(0, 2, 1))
x_r

RuntimeError: Expected batch2_sizes[0] == bs && batch2_sizes[1] == contraction_size to be true, but got false.  (Could this error message be improved?  If so, please report an enhancement request to PyTorch.)

In [3]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    attn_logits = torch.matmul(q, k.transpose(-2, -1))
    attn_logits = attn_logits / math.sqrt(d_k)
    if mask is not None:
        attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
    attention = F.softmax(attn_logits, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [8]:
import pytorch_lightning as pl
import math
import torch.nn.functional as F
seq_len, d_k = 3, 2
# pl.seed_everything(42)
q = torch.randn(seq_len, d_k)
k = torch.randn(seq_len, d_k)
v = torch.randn(seq_len, d_k)
values, attention = scaled_dot_product(q, k, v)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("Values\n", values)
print("Attention\n", attention)

Q
 tensor([[-0.3655,  0.7285],
        [ 0.6512,  2.0699],
        [-1.4464,  0.2941]])
K
 tensor([[ 0.2889,  1.9247],
        [-1.2538, -1.0563],
        [-0.1256,  0.4617]])
V
 tensor([[-0.2521,  1.1885],
        [ 0.8997,  0.3598],
        [ 1.8922,  0.0591]])
Values
 tensor([[ 0.5572,  0.7237],
        [-0.0568,  1.0844],
        [ 0.8927,  0.4633]])
Attention
 tensor([[0.5421, 0.1739, 0.2840],
        [0.9063, 0.0057, 0.0880],
        [0.2113, 0.5506, 0.2381]])


In [103]:
class CrossedAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.q_conv = nn.Conv1d(1024, 256, 1, bias=False)
        self.k_conv = nn.Conv1d(1024, 256, 1, bias=False)
        self.v_conv = nn.Conv1d(1024, 1024, 1, bias=False)
        self.trans_conv = nn.Conv1d(1024, 1024, 1)
        self.after_norm = nn.BatchNorm1d(1024)
        self.act = nn.ReLU()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q_tensor, kv_tensor):
        print('q_tensor: ', q_tensor.shape)
        x_q = self.q_conv(q_tensor.permute(0, 2, 1))    
        x_k = self.k_conv(kv_tensor.permute(0, 2, 1))  
        x_v = self.v_conv(kv_tensor.permute(0, 2, 1))
        print('x_q: ', x_q.shape)
        energy = torch.matmul(x_q.permute(0, 2, 1), x_k)
        print('energy shape: ', energy.shape)
        attention = self.softmax(energy)
        # attention = attention / (1e-9 + attention.sum(dim=1, keepdims=True))
        x_r = torch.matmul(attention, x_v.permute(0, 2, 1))  
        print('x_r shape: ', x_r.shape)
        
        res = (q_tensor - x_r).permute(0, 2, 1)
        print('res shape: ', res.shape)
        
        x_r = self.act(self.after_norm(self.trans_conv(res)))
        x_r = x_r.permute(0, 2, 1) + q_tensor
        print('result shape: ', x_r.shape)
        
        return x_r
    
cb = CrossedAttention(1024)


In [104]:
rand_q = torch.ones((4, 1, 1024))
rand_kv = torch.ones((4, 6, 1024))

result = cb(rand_q, rand_kv)

q_tensor:  torch.Size([4, 1, 1024])
x_q:  torch.Size([4, 256, 1])
energy shape:  torch.Size([4, 1, 6])
x_r shape:  torch.Size([4, 1, 1024])
res shape:  torch.Size([4, 1024, 1])
result shape:  torch.Size([4, 1, 1024])
